In [18]:
#!pip install ../input/kerasapplications/ > /dev/null
#!pip install ../input/efficientnet-keras-source-code/ > /dev/null

# **Libraries**

In [19]:
import gc
import os
import math
import random
import re
import warnings
from pathlib import Path
from PIL import Image
from typing import Optional, Tuple
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import spatial
from sklearn.preprocessing import normalize
from tqdm import tqdm

## Settings

## Utilities

In [20]:
import time

from contextlib import contextmanager


@contextmanager
def timer(name):
    t0 = time.time()
    print(f"[{name}]")
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

## Model

## Feature Extraction

In [21]:
def feature_extract(root_dir, n_models=4):
    #return ids, embeddings
    pass

## Main

In [22]:
DIRFILE = '../input/landmark-retrieval-2021/'

index_data = f'{DIRFILE}/index'
train_data_dir = f'{DIRFILE}/train'
test_data_dir = f'{DIRFILE}/test'

train = pd.read_csv(f'{DIRFILE}/train.csv')
#sub = pd.read_csv(f'{DIRFILE}/train.csv')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
def get_predictions():
    with timer("Getting Test Embeddings"):
        test_ids, test_embeddings = feature_extract(str(test_data_dir))
        
    with timer("Getting Train Embeddings"):
        train_ids, train_embeddings = feature_extract(str(train_data_dir))
        
    Prediction_List = []
    
    with timer('Matching...'):
        for test_index in range(test_embeddings.shape[0]):
            distances = spatial.distances.cdist(test_embeddings[np.newaxis, test_index, :],
                                                train_embeddings, 'cosine')[0]
            part = np.argpartition(distances, TOPK)[:TOPK]
            nearest_matches = sorted([(train_ids[p], distances[p]) for p in part], key=lambda x:x[1])
            pred_str = ""
            for train_ids, cosine_dist in nearest_matches:
                pred_str += train_id
                pred_str += " "
            Prediction_List.append(pred_str)
            
    return test_index, Prediction_List
    
        

def main():
    test_image_list = []
    for root, dirs, files in os.walk(str(test_data_dir)):
        for file in files:
            if file.endswith('.jpg'):
                test_image_list.append(os.path.join(root,file))
                
    if len(test_image_list) == 1129:
        sub_df = pd.read_csv('../input/land-mark-retrieval-2021/sample_submission.csv')
        sub_df.to_csv('submission.csv', index=False)
        return
    
    test_ids, PredictionStringList = get_predictions()
    
    sub_df = pd.DataFrame(data = {'id': test_ids, 'images':PredictionStringList})
    sub_df.to_csv('submission.csv', index=False)

This notebook is based on [DELG_global_baseline](https://www.kaggle.com/wowfattie/delg-global-baseline).